<a href="https://colab.research.google.com/github/secutron/ShowTime/blob/master/up_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Candel, A., Parmar, V., LeDell, E., and Arora, A. (Jun 2020). Deep Learning
with H2O. http://h2o.ai/resources.

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/booklets/DeepLearningBooklet.pdf?_ga=2.184768361.1553179602.1592730801-433090902.1592730801



In [1]:
!nvidia-smi

Tue Jun 23 06:11:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 128.4MB 95kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.5-py2.py3-none-any.whl size=128410595 sha256=ae33b7d7aff5359f818af0acac5050bb57e973176ba69197d7c9605f119225d0
  Stored in directory: /root/.cache/pip/wheels/52/ac/34/be36def52a75ce00aadf2312a66db43e1bb6226e0d4cc51905
Successfully built h2o


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import urllib
import h2o
from h2o.automl import H2OAutoML
import sys
import os

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
print('matplotlib: {}'.format(matplotlib.__version__))
#print('sklearn: {}'.format(sklearn.__version__))
#print('scipy: {}'.format(scipy.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('sys: {}'.format(sys.version))

matplotlib: 3.2.2
seaborn: 0.10.1
pandas: 1.0.5
numpy: 1.18.5
sys: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpws0fxkme
  JVM stdout: /tmp/tmpws0fxkme/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpws0fxkme/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_unknownUser_f6ebon
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
# Get help
#help(h2o.estimators.glm.H2OGeneralizedLinearEstimator)
#help(h2o.estimators.gbm.H2OGradientBoostingEstimator)
#help(h2o.estimators.deeplearning.H2ODeepLearningEstimator)

# Show a demo
#h2o.demo("glm")
#h2o.demo("gbm")
#h2o.demo("deeplearning")

### 여러 파일 준비

In [8]:
BASE_DIR = '/content'
#OUTPUT_FILE = os.path.join(BASE_DIR, 'tmp.csv')

#hkim_data=urllib.request.urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', OUTPUT_FILE)

# 드라이브 전체공유 후..

# column_2C_weka.csv
#hkim_data=urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1oMEvI6gWC4ZAqssQ3s3j6RkWUPa_TSwY', OUTPUT_FILE)

#churn_df = h2o.import_file('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [9]:
OUTPUT_FILE = os.path.join(BASE_DIR, 'QMS_merge_sort_ALL.csv')
df = pd.read_csv(OUTPUT_FILE, delimiter = ',')
df.shape

(266667, 31)

In [10]:
df.head()

,Unnamed: 0,a1.ucube_jnr_no,a2.firstlogtime,a2.lastlogtime,a2.errtype,a2.errtype_nm,a2.cust_num,a2.model_mac,a2.buffer_overflow,a2.pcr_lock_status,a2.pts_advanced_error,a2.lost_packet_count,a2.packet_loss_period,a2.stb_rtp_errcnt,a2.no_stream_error,a2.stb_firmware_version,a2.no_stream_error_sum,a2.pcr_lock_status_sum,a2.pts_advanced_error_sum,a2.stb_rtp_errcnt_sum,a2.logtime,a2.not_decoding_error,a2.not_decoding_error_sum,a2.mday,a2.bdp_load_dttm,a2.bdp_load_user_id,a2.p_basis_yyyy,a2.p_basis_mm,a2.p_basis_dd,a2.p_basis_hh,a2.model_nm
0,761343,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:00.000000,2020-06-01 05:00:00.000000,26.0,1일 1회 주기로 최신 업데이트된 리모컨 진단 정보 전송,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01 05:00:00.000000,2020-06-01 05:15:41.676770,NaN,NaN,NaN,NaN,NaN,S60UPI
1,633634,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:01.000000,2020-06-01 05:00:01.000000,16.0,SLEEP To Active 모드 전환,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01 05:00:01.000000,2020-06-01 05:15:41.676770,NaN,NaN,NaN,NaN,NaN,S60UPI
2,761344,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:02.000000,2020-06-01 05:00:02.000000,31.0,HDMI 연결상태 변경,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01 05:00:02.000000,2020-06-01 05:15:41.676770,NaN,NaN,NaN,NaN,NaN,S60UPI
3,761345,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:13.000000,2020-06-01 05:00:13.000000,31.0,HDMI 연결상태 변경,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01 05:00:14.000000,2020-06-01 05:15:41.676770,NaN,NaN,NaN,NaN,NaN,S60UPI
4,633635,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:08:45.000000,2020-06-01 17:08:45.000000,16.0,SLEEP To Active 모드 전환,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01 17:08:45.000000,2020-06-01 17:15:47.098461,NaN,NaN,NaN,NaN,NaN,S60UPI


In [11]:
hdf = h2o.import_file(OUTPUT_FILE)
hdf.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,a1.ucube_jnr_no,a2.firstlogtime,a2.lastlogtime,a2.errtype,a2.errtype_nm,a2.cust_num,a2.model_mac,a2.buffer_overflow,a2.pcr_lock_status,a2.pts_advanced_error,a2.lost_packet_count,a2.packet_loss_period,a2.stb_rtp_errcnt,a2.no_stream_error,a2.stb_firmware_version,a2.no_stream_error_sum,a2.pcr_lock_status_sum,a2.pts_advanced_error_sum,a2.stb_rtp_errcnt_sum,a2.logtime,a2.not_decoding_error,a2.not_decoding_error_sum,a2.mday,a2.bdp_load_dttm,a2.bdp_load_user_id,a2.p_basis_yyyy,a2.p_basis_mm,a2.p_basis_dd,a2.p_basis_hh,a2.model_nm
761343,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:00,2020-06-01 05:00:00,26,1일 1회 주기로 최신 업데이트된 리모컨 진단 정보 전송,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:00,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
633634,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:01,2020-06-01 05:00:01,16,SLEEP To Active 모드 전환,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:01,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
761344,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:02,2020-06-01 05:00:02,31,HDMI 연결상태 변경,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:02,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
761345,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:13,2020-06-01 05:00:13,31,HDMI 연결상태 변경,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:14,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
633635,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:08:45,2020-06-01 17:08:45,16,SLEEP To Active 모드 전환,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:08:45,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI
761346,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:08:45,2020-06-01 17:08:45,31,HDMI 연결상태 변경,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:08:46,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI
761347,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:08:51,2020-06-01 17:08:51,38,채널 전환 지연,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:08:52,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI
633636,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:09:11,2020-06-01 17:09:11,11,ethernet port up,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:09:11,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI
633637,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:09:11,2020-06-01 17:09:11,12,DHCP 서버로부터 IP 획득 성공,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:09:11,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI
761348,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 17:09:26,2020-06-01 17:09:26,22,BT 연결 CONNECTED,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 17:09:26,2020-06-01 17:15:47,,nan,nan,nan,nan,S60UPI


In [12]:
#df.types
hdf.types

{'C1': 'int',
 'a1.ucube_jnr_no': 'enum',
 'a2.bdp_load_dttm': 'time',
 'a2.bdp_load_user_id': 'enum',
 'a2.buffer_overflow': 'int',
 'a2.cust_num': 'enum',
 'a2.errtype': 'int',
 'a2.errtype_nm': 'enum',
 'a2.firstlogtime': 'time',
 'a2.lastlogtime': 'time',
 'a2.logtime': 'int',
 'a2.lost_packet_count': 'int',
 'a2.mday': 'time',
 'a2.model_mac': 'enum',
 'a2.model_nm': 'enum',
 'a2.no_stream_error': 'int',
 'a2.no_stream_error_sum': 'int',
 'a2.not_decoding_error': 'int',
 'a2.not_decoding_error_sum': 'int',
 'a2.p_basis_dd': 'int',
 'a2.p_basis_hh': 'int',
 'a2.p_basis_mm': 'int',
 'a2.p_basis_yyyy': 'int',
 'a2.packet_loss_period': 'int',
 'a2.pcr_lock_status': 'int',
 'a2.pcr_lock_status_sum': 'int',
 'a2.pts_advanced_error': 'int',
 'a2.pts_advanced_error_sum': 'int',
 'a2.stb_firmware_version': 'enum',
 'a2.stb_rtp_errcnt': 'int',
 'a2.stb_rtp_errcnt_sum': 'int'}

In [13]:
df.describe()

,Unnamed: 0,a2.errtype,a2.buffer_overflow,a2.pcr_lock_status,a2.pts_advanced_error,a2.lost_packet_count,a2.packet_loss_period,a2.stb_rtp_errcnt,a2.no_stream_error,a2.no_stream_error_sum,a2.pcr_lock_status_sum,a2.pts_advanced_error_sum,a2.stb_rtp_errcnt_sum,a2.logtime,a2.not_decoding_error,a2.not_decoding_error_sum,a2.p_basis_yyyy,a2.p_basis_mm,a2.p_basis_dd,a2.p_basis_hh
count,2.666670e+05,245643.000000,21023.000000,21023.000000,21023.000000,21023.0,21023.0,21023.000000,21023.000000,21023.000000,21023.000000,21023.000000,21023.000000,2.102300e+04,21023.000000,21023.000000,21023.0,21023.0,21023.000000,21023.000000
mean,9.715956e+05,19.023652,47.850782,-0.234981,47.850782,0.0,0.0,196.836465,1.946915,26.308567,0.168244,577.226657,2350.651667,2.020060e+13,-0.246587,0.027399,2020.0,6.0,4.518242,13.199115
std,6.207238e+05,10.070516,2410.002135,0.502074,2410.002135,0.0,0.0,3166.810292,28.075805,252.965569,1.273955,15456.922043,21641.751755,2.263659e+06,0.440855,0.345143,0.0,0.0,2.271513,7.016035
min,4.560000e+02,1.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.020060e+13,-1.000000,0.000000,2020.0,6.0,1.000000,0.000000
25%,4.785105e+05,10.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.020060e+13,0.000000,0.000000,2020.0,6.0,3.000000,8.000000
50%,9.158650e+05,16.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,2.020061e+13,0.000000,0.000000,2020.0,6.0,5.000000,14.000000
75%,1.551892e+06,31.000000,0.000000,0.000000,0.000000,0.0,0.0,4.000000,0.000000,0.000000,0.000000,0.000000,327.000000,2.020061e+13,0.000000,0.000000,2020.0,6.0,7.000000,19.000000
max,2.028312e+06,41.000000,158862.000000,13.000000,158862.000000,0.0,0.0,128874.000000,600.000000,7200.000000,29.000000,556292.000000,509630.000000,2.020061e+13,7.000000,10.000000,2020.0,6.0,9.000000,23.000000


In [14]:
hdf.describe()

Rows:266667
Cols:31




,C1,a1.ucube_jnr_no,a2.firstlogtime,a2.lastlogtime,a2.errtype,a2.errtype_nm,a2.cust_num,a2.model_mac,a2.buffer_overflow,a2.pcr_lock_status,a2.pts_advanced_error,a2.lost_packet_count,a2.packet_loss_period,a2.stb_rtp_errcnt,a2.no_stream_error,a2.stb_firmware_version,a2.no_stream_error_sum,a2.pcr_lock_status_sum,a2.pts_advanced_error_sum,a2.stb_rtp_errcnt_sum,a2.logtime,a2.not_decoding_error,a2.not_decoding_error_sum,a2.mday,a2.bdp_load_dttm,a2.bdp_load_user_id,a2.p_basis_yyyy,a2.p_basis_mm,a2.p_basis_dd,a2.p_basis_hh,a2.model_nm
type,int,enum,time,time,int,enum,enum,enum,int,int,int,int,int,int,int,enum,int,int,int,int,int,int,int,time,time,enum,int,int,int,int,enum
mins,456.0,,1590969600000.0,1590969600000.0,1.0,,,,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,,0.0,0.0,0.0,0.0,20200601001210.0,-1.0,0.0,1590969601000.0,1590970543329.0,,2020.0,6.0,1.0,0.0,
mean,971595.5937217585,,1591305220839.614,1591305279797.789,19.023652210728557,,,,47.85078247633539,-0.2349807353850543,47.85078247633539,0.0,0.0,196.8364648242401,1.9469152832611902,,26.308566807781943,0.1682443038576797,577.2266565190504,2350.6516672216108,20200604643472.04,-0.24658707130285887,0.027398563478095418,1591305530892.878,1591306199672.2012,,2020.0,6.0,4.51824192551016,13.199115254721015,
maxs,2028312.0,,1591660706000.0,1591661305000.0,41.0,,,,158862.0,13.0,158862.0,0.0,0.0,128874.0,600.0,,7200.0,29.0,556292.0,509630.0,20200609015557.0,7.0,10.0,1591667400000.0,1591668084758.0,,2020.0,6.0,9.0,23.0,
sigma,620723.7878983024,,192976740.1013083,192983474.18581322,10.070516262524144,,,,2410.0021353721977,0.5020742422435859,2410.0021353721977,0.0,0.0,3166.810291913966,28.075804902506526,,252.96556876471283,1.273955495969517,15456.922042706195,21641.751755410627,2263659.1955774967,0.44085505945185166,0.34514301425229094,192987495.80629304,192993573.72853816,,0.0,0.0,2.271513437263717,7.016034557808338,
zeros,0,,0,0,0,,,,14900,15668,14900,21023,21023,7721,15385,,18482,20019,19631,574,0,15761,20747,0,0,,0,0,0,1376,
missing,0,0,1,1,21024,21024,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,245644,1,1,245644,245644,245644,245644,245644,21024
0,761343.0,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:00,2020-06-01 05:00:00,26.0,1일 1회 주기로 최신 업데이트된 리모컨 진단 정보 전송,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:00,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
1,633634.0,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:01,2020-06-01 05:00:01,16.0,SLEEP To Active 모드 전환,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:01,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI
2,761344.0,0016CF82DEDAE7D04E204280145D53AC20D56E87,2020-06-01 05:00:02,2020-06-01 05:00:02,31.0,HDMI 연결상태 변경,,,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,2020-06-01 05:00:02,2020-06-01 05:15:41,,nan,nan,nan,nan,S60UPI


In [15]:
data_train,data_test,data_valid = hdf.split_frame(ratios=[.7, .15])

In [16]:
y = "a2.errtype"
x = hdf.columns
x.remove(y)
x.remove("C1")
x.remove("a1.ucube_jnr_no")
x.remove("a2.errtype_nm")
x.remove("a2.firstlogtime")
x.remove("a2.lastlogtime")
x.remove("a2.mday")
x.remove("a2.bdp_load_dttm")
x.remove("a2.logtime")
x.remove("a2.p_basis_yyyy")
x.remove("a2.p_basis_mm")
x.remove("a2.p_basis_dd")
x.remove("a2.p_basis_hh")
# a2.p_basis_yyyy	a2.p_basis_mm	a2.p_basis_dd	a2.p_basis_hh	
x

['a2.cust_num',
 'a2.model_mac',
 'a2.buffer_overflow',
 'a2.pcr_lock_status',
 'a2.pts_advanced_error',
 'a2.lost_packet_count',
 'a2.packet_loss_period',
 'a2.stb_rtp_errcnt',
 'a2.no_stream_error',
 'a2.stb_firmware_version',
 'a2.no_stream_error_sum',
 'a2.pcr_lock_status_sum',
 'a2.pts_advanced_error_sum',
 'a2.stb_rtp_errcnt_sum',
 'a2.not_decoding_error',
 'a2.not_decoding_error_sum',
 'a2.bdp_load_user_id',
 'a2.model_nm']

#Create model

In [17]:
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)

# Train model

In [18]:
aml.train(x = x, y = y, training_frame = data_train, validation_frame=data_valid)
!nvidia-smi

AutoML progress: |
06:12:41.171: Project: AutoML_20200623_61241167
06:12:41.171: Cross-validation disabled by user: no fold column nor nfolds > 1.
06:12:41.172: Setting stopping tolerance adaptively based on the training frame: 0.002314095739546091
06:12:41.173: Build control seed: 10
06:12:41.174: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
06:12:42.601: training frame: Frame key: automl_training_py_3_sid_bbfe    cols: 31    rows: 167794  chunks: 8    size: 8617506  checksum: 9181734551656143523
06:12:42.601: validation frame: Frame key: py_5_sid_bbfe    cols: 31    rows: 39896  chunks: 8    size: 2185562  checksum: 4726372329276196540
06:12:42.619: leaderboard frame: Frame key: automl_leaderboard_py_3_sid_bbfe    cols: 31    rows: 18946  chunks: 8    size: 1136378  checksum: -558670567243359916
06:12:42.619: blending frame: NULL
06:12:42.619: response column: a

In [19]:
leaderboard = aml.leaderboard
leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20200623_061241,73.8732,8.59495,73.8732,6.98075,0.496011
GBM_grid__1_AutoML_20200623_061241_model_1,73.8742,8.59501,73.8742,6.98153,0.496067
GBM_1_AutoML_20200623_061241,73.8772,8.59519,73.8772,6.97937,0.496213
GBM_4_AutoML_20200623_061241,73.8811,8.59541,73.8811,6.97972,0.496375
GBM_grid__1_AutoML_20200623_061241_model_2,73.8831,8.59553,73.8831,6.97948,0.496302
GBM_3_AutoML_20200623_061241,73.8843,8.5956,73.8843,6.97934,0.496412
GBM_5_AutoML_20200623_061241,73.8848,8.59563,73.8848,6.97946,0.496412
GBM_grid__1_AutoML_20200623_061241_model_3,73.8867,8.59573,73.8867,6.97897,0.49646
DRF_1_AutoML_20200623_061241,87.9094,9.376,87.9094,8.12304,0.551883
GLM_1_AutoML_20200623_061241,101.402,10.0698,101.402,8.8695,0.597914


#Test

In [20]:
data_pred=aml.leader.predict(data_test)
data_pred.head()

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147


# Performance check


In [21]:
aml.leader.model_performance(data_test)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 72.84118085870979
RMSE: 8.53470449744511
MAE: 6.94945167113356
RMSLE: 0.49128152017050175
Mean Residual Deviance: 72.84118085870979


#나머지 모델도 테스트

In [22]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
model_ids
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
#metalearner = h2o.get_model(se.metalearner()['name'])

['GBM_2_AutoML_20200623_061241',
 'GBM_grid__1_AutoML_20200623_061241_model_1',
 'GBM_1_AutoML_20200623_061241',
 'GBM_4_AutoML_20200623_061241',
 'GBM_grid__1_AutoML_20200623_061241_model_2',
 'GBM_3_AutoML_20200623_061241',
 'GBM_5_AutoML_20200623_061241',
 'GBM_grid__1_AutoML_20200623_061241_model_3',
 'DRF_1_AutoML_20200623_061241',
 'GLM_1_AutoML_20200623_061241']

In [27]:
h2o.get_model([i for i in model_ids if "GBM" in i][0])

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_2_AutoML_20200623_061241


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,65.0,65.0,6174.0,1.0,3.0,1.707692,1.0,4.0,2.830769




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 73.43926943061557
RMSE: 8.56967148907212
MAE: 6.961297096347723
RMSLE: 0.49317667165871293
Mean Residual Deviance: 73.43926943061557

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 73.22238708734677
RMSE: 8.557008068673698
MAE: 6.948941820497263
RMSLE: 0.49289179508677106
Mean Residual Deviance: 73.22238708734677

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2020-06-23 06:13:04,0.001 sec,0.0,10.078007,8.877178,101.566233,10.067944,8.868223,101.363495
1,,2020-06-23 06:13:05,0.654 sec,5.0,9.441938,8.157981,89.150200,9.431384,8.152456,88.950996
2,,2020-06-23 06:13:06,1.256 sec,10.0,9.151202,7.780918,83.744501,9.140411,7.776521,83.547119
3,,2020-06-23 06:13:06,1.677 sec,15.0,8.955238,7.508384,80.196291,8.944177,7.504135,79.998300
4,,2020-06-23 06:13:07,2.187 sec,20.0,8.737216,7.209245,76.338946,8.725673,7.203988,76.137373
5,,2020-06-23 06:13:07,2.468 sec,25.0,8.687592,7.152174,75.474247,8.675832,7.146256,75.270069
6,,2020-06-23 06:13:07,2.742 sec,30.0,8.647056,7.100957,74.771579,8.635128,7.094341,74.565430
7,,2020-06-23 06:13:08,3.232 sec,35.0,8.597908,7.014813,73.924019,8.585654,7.006396,73.713450
8,,2020-06-23 06:13:08,3.821 sec,40.0,8.579533,6.967772,73.608382,8.567040,6.957477,73.394168
9,,2020-06-23 06:13:09,4.244 sec,45.0,8.573818,6.953583,73.510362,8.561260,6.942438,73.295181



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,a2.model_nm,21164592.0,1.0,1.0
1,a2.cust_num,0.0,0.0,0.0
2,a2.model_mac,0.0,0.0,0.0
3,a2.buffer_overflow,0.0,0.0,0.0
4,a2.pcr_lock_status,0.0,0.0,0.0
5,a2.pts_advanced_error,0.0,0.0,0.0
6,a2.lost_packet_count,0.0,0.0,0.0
7,a2.packet_loss_period,0.0,0.0,0.0
8,a2.stb_rtp_errcnt,0.0,0.0,0.0
9,a2.no_stream_error,0.0,0.0,0.0


In [29]:
gbm = h2o.get_model([i for i in model_ids if "GBM" in i][0])
gbm.summary()


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,65.0,65.0,6174.0,1.0,3.0,1.707692,1.0,4.0,2.830769


In [30]:
gbm.model_performance(data_valid)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 73.22238708734677
RMSE: 8.557008068673698
MAE: 6.948941820497263
RMSLE: 0.49289179508677106
Mean Residual Deviance: 73.22238708734677


In [56]:
pred_val = gbm.predict(data_valid[x])
pred_val.head(20)


gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147
21.5147


In [54]:
true_val = data_valid[y]
true_val.head(20)


a2.errtype
16
38
11
26
41
5
23
16
26
31


In [50]:
pred_val = list(gbm.predict(data_valid[x])[0])
true_val = list(data_valid[y])

tmp = pd.DataFrame(pred_val)



gbm prediction progress: |████████████████████████████████████████████████| 100%


In [ ]:

prediction_acc = np.mean(pred_val == true_val)
print("Prediction accuracy: ", prediction_acc)

In [23]:
drf = h2o.get_model([i for i in model_ids if "DRF" in i][0])

In [24]:
#h2o.get_model([i for i in model_ids if "DeepLearning" in i][0])

IndexError: ignored

In [ ]:
dl = h2o.get_model([i for i in model_ids if "DeepLearning" in i][0])

In [ ]:
dl.confusion_matrix()

In [ ]:
dl.varimp_plot()

#모델 저장

In [ ]:
aml.leader.download_mojo(path = BASE_DIR)